In [ ]:
import re
import datetime
import ast
from collections import Counter
from pprint import pprint

import business_models
import tqdm.notebook
import pandas as pd
import py3dbp
import numpy as np
from nile.api.v1 import (
    filters as nf,
    aggregators as na,
    extractors as ne,
    Record
)
from qb2.api.v1 import (filters as qf, typing)

from projects.common.nile import dates
from projects.common.nile import test_utils
from projects.k2.adhocs import project_config, default_values, common

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=10,
)

### Define constants

In [ ]:
DT_START = '2021-04-26'
DT_END = '2021-05-13'

PROJECT_PATH = '//home/taxi-delivery/analytics/dev/product/postpayment/'
APPMETRICA_TAXIMETER = '//home/taxi-dwh/ods/appmetrica/taximeter/{}'
DRIVER_CUBE = '//home/taxi-analytics/d-captain/driver_cube_v3/{}'

### Appmetrica

### Воронка

#### Общее
- сколько активных в регионе
- сколько активных в регионе с версией 9.76 и выше (с тарифной опцией)
- сколько активных в регионе с версией 9.76 и выше и с NFC
- сколько активных в регионе с версией 9.76 и выше и без NFC

#### Обучение
- сколько активных открывших обучение
- сколько активных начавших обучение
- сколько активных включивших опцию в опциях

#### Диагностика
- сколько активных с NFC, включивших опцию и дошедших до NFC
- сколько активных с NFC, включивших опцию и включивших NFC
- сколько активных с NFC, включивших NFC и дошедших до tap2go
- сколько активных с NFC, включивших NFC и скачавших tap2go

#### Выход на линию
- сколько активных вышедших на линию с опцией
- сколько активных вышедших на линию без опции

### Флаги
- активный
- активный в регине
- версия 9.76 и выше
- андроид 9 и выше
- есть NFC
- открыл обучение
- начал обучение
- включил тарифную опцию
- дошел до включите NFC
- включил NFC
- дошел до tap2go
- скачал tap2go
- вышел на линию

In [ ]:
APP_VERSION_FIRST_RELEASE = 9.76
ANDROID_VERSION = 9
REGION_IDS = {54,}

In [ ]:
def to_type(obj, target_type):
    try:
        return target_type(obj)
    except:
        return obj

In [ ]:
def executor_reducer(groups):
    for key, records in groups:
        key_dict = test_utils.to_string(key.to_dict())
        executor_dict = {}
        executor_dict.update(key_dict)
        
        record_list = []
        for record in records:
            record_dict = test_utils.to_string(record.to_dict())
            record_list.append(record_dict)
            
        driver_options_list = list(filter(
            lambda x: x.get('event_name') == 'driver_options',
            record_list
        ))
        ui_events_list = list(filter(
            lambda x: x.get('event_name') == 'ui_event',
            record_list
        ))
        final_button_clicked_list = list(filter(
            lambda x: x.get('event_name') == 'FinalButtonClicked',
            record_list
        ))
        online_events = list(filter(
            lambda x: x.get('event_name') == 'tabbar/status/online',
            record_list
        ))
        
        # App and platform flags.
        executor_dict['active'] = True
        executor_dict['first_event_dttm'] = (
            min([r.get('utc_event_dttm') for r in record_list])
            if record_list else None
        )
        test_region_record_list = list(filter(lambda x: x.get('region_id') in REGION_IDS, record_list))
        executor_dict['active_test_region'] = bool(test_region_record_list)
        executor_dict['first_test_region_event_dttm'] = (
            min([r.get('utc_event_dttm') for r in test_region_record_list])
            if test_region_record_list else None
        )
        if executor_dict['active_test_region']:
            record_list = test_region_record_list
        executor_dict['correct_app_version_flg'] = (
            max([to_type(r.get('app_version_name', '0'), float)
                 if isinstance(to_type(r.get('app_version_name', '0'), float), float)
                 else 0.
                 for r in record_list])
            >= APP_VERSION_FIRST_RELEASE
        )
        executor_dict['correct_android_version_flg'] = (
            max([to_type(r.get('os_version', '0')[:2], float)
                 if isinstance(r.get('os_version', '0'), str)
                 and isinstance(to_type(r.get('os_version', '0')[:2], float), float)
                 else 0.
                 for r in record_list])
            >= ANDROID_VERSION
        )
        executor_dict['has_nfc_flg'] = None
        
        # Exam flags.
        seen_exam_events = [
            r for r in driver_options_list if
            r.get('event_value', {}).get('driveroptionsmetricaparams', {}).get('option_id')=='post_payment'
            and r.get('event_value', {}).get('driveroptionsmetricaparams', {}).get('action')=='show_exam_dialog'
        ]
        executor_dict['seen_exam_flg'] = bool(seen_exam_events)
        executor_dict['first_seen_exam_dttm'] = (
            min([r.get('utc_event_dttm') for r in seen_exam_events])
            if seen_exam_events else None
        )
        opened_exam_events = [
            r for r in driver_options_list if
            r.get('event_value', {}).get('driveroptionsmetricaparams', {}).get(
                'option_id')=='post_payment'
            and r.get('event_value', {}).get('driveroptionsmetricaparams', {}).get(
                'action')=='open_exam_link'
        ]
        executor_dict['opened_exam_flg'] = bool(opened_exam_events)
        executor_dict['first_opened_exam_dttm'] = (
            min([r.get('utc_event_dttm') for r in opened_exam_events])
            if opened_exam_events else None
        )
        switched_option_on_events = [
            r for r in driver_options_list if
            r.get('event_value', {}).get('driveroptionsmetricaparams', {}).get(
                'option_id')=='post_payment'
            and r.get('event_value', {}).get('driveroptionsmetricaparams', {}).get(
                'action')=='switch'
            and r.get('event_value', {}).get('driveroptionsmetricaparams', {}).get(
                'option_value')==True
        ]
        executor_dict['switched_option_on_flg'] = bool(switched_option_on_events)
        executor_dict['first_switched_option_on_dttm'] = (
            min([r.get('utc_event_dttm') for r in switched_option_on_events])
            if switched_option_on_events else None
        )
        
        # NFC and Tap2Go flags.
        no_nfc_screen_shown_events = [
            r for r in ui_events_list if
            r.get('event_value', {}).get('diagnosticcenterparams', {}).get(
                'screentag')=='PostPaymentNFCOff'
            and r.get('event_value', {}).get('uiparams', {}).get(
                'uicomponentname')=='ScreenShown'
        ]
        executor_dict['no_nfc_screen_shown_flg'] = bool(no_nfc_screen_shown_events)
        executor_dict['first_no_nfc_screen_shown_dttm'] = (
            min([r.get('utc_event_dttm') for r in no_nfc_screen_shown_events])
            if no_nfc_screen_shown_events else None
        )
        
        no_nfc_button_clicked_events = [
            r for r in final_button_clicked_list if
            r.get('event_value', {}).get('diagnosticcenterparams', {}).get(
                'screentag')=='PostPaymentNFCOff'
        ]
        executor_dict['no_nfc_button_clicked_flg'] = bool(no_nfc_button_clicked_events)
        executor_dict['first_no_nfc_button_clicked_dttm'] = (
            min([r.get('utc_event_dttm') for r in no_nfc_button_clicked_events])
            if no_nfc_button_clicked_events else None
        )
        
        no_tap2go_screen_shown_events = [
            r for r in ui_events_list if
            r.get('event_value', {}).get('diagnosticcenterparams', {}).get(
                'screentag')=='PostPaymentNoApp'
            and r.get('event_value', {}).get('uiparams', {}).get(
                'uicomponentname')=='ScreenShown'
        ]
        executor_dict['no_tap2go_screen_shown_flg'] = bool(no_tap2go_screen_shown_events)
        executor_dict['first_no_tap2go_screen_shown_dttm'] = (
            min([r.get('utc_event_dttm') for r in no_tap2go_screen_shown_events])
            if no_tap2go_screen_shown_events else None
        )
        
        no_tap2go_button_clicked_events = [
            r for r in final_button_clicked_list if
            r.get('event_value', {}).get('diagnosticcenterparams', {}).get(
                'screentag')=='PostPaymentNoApp'
        ]
        executor_dict['no_tap2go_button_clicked_flg'] = bool(no_tap2go_button_clicked_events)
        executor_dict['first_no_tap2go_button_clicked_dttm'] = (
            min([r.get('utc_event_dttm') for r in no_tap2go_button_clicked_events])
            if no_tap2go_button_clicked_events else None
        )
        
        # Success line flag.    
        executor_dict['online_flg'] = bool(online_events)
        executor_dict['first_online_dttm'] = (
            min([r.get('utc_event_dttm') for r in online_events])
            if online_events else None
        )
        executor_dict['last_online_dttm'] = (
            max([r.get('utc_event_dttm') for r in online_events])
            if online_events else None
        )
        executor_dict['first_online_before_option_switch_flg'] = (
            executor_dict['first_online_dttm'] < executor_dict['first_switched_option_on_dttm']
            if executor_dict['first_online_dttm'] and executor_dict['first_switched_option_on_dttm']
            else False
        )
        executor_dict['last_online_after_option_switch_flg'] = (
            executor_dict['last_online_dttm'] >= executor_dict['first_switched_option_on_dttm']
            if executor_dict['last_online_dttm'] and executor_dict['first_switched_option_on_dttm']
            else False
        )
        
        yield Record.from_dict(executor_dict)

In [ ]:
# key = Record.from_dict({'executor_unique_id': 'boy_next_door', 'utc_event_dt': '2021-04-26'})
# records = [
#     Record.from_dict(dict(item._asdict()))
#     for item in df.sort_values('utc_event_dttm').itertuples()
# ]
# groups = ((key, records),)

# next(executor_reducer(groups)).to_dict()

In [ ]:
job = cluster.job()

In [ ]:
appmetrica = job.table(
    APPMETRICA_TAXIMETER.format(dates.range_selector(DT_START, DT_END)),
    ignore_missing=True
)

driver_cube = job.table(
    DRIVER_CUBE.format(dates.range_selector(DT_START, DT_END, '%Y-%m-01')),
    ignore_missing=True
)

logistics_couriers = (
    driver_cube
    .filter(
        nf.or_(
            nf.custom(
                lambda x: 'walking_courier' in to_type(test_utils.to_string(x), dict)
                if isinstance(to_type(test_utils.to_string(x), dict), dict) else False,
                'tag'
            ),
            nf.custom(
                lambda x: to_type(test_utils.to_string(x), dict).get('auto_courier', 0) >= 0.8
                if isinstance(to_type(test_utils.to_string(x), dict), dict) else False,
                'tag'
            ),
        ),
        nf.custom(
            lambda x: DT_START <= str(datetime.date.fromtimestamp(x)) <= DT_END
            if x else False,
            'dt'
        ),
        qf.defined('dbid_uuid', 'dt',),
    )
    .project(
        'dbid_uuid',
        utc_dt=ne.custom(lambda x: str(datetime.date.fromtimestamp(x)), 'dt'),
        logistics_courier_flg=ne.const(True)
    )
    .groupby('dbid_uuid', 'utc_dt',)
    .aggregate(logistics_courier=na.max('logistics_courier_flg'))
)

(
    appmetrica
    .filter(
        nf.equals('app_platform', b'android'),
        qf.defined('executor_profile_id', 'park_taximeter_id', 'utc_event_dttm',),
        nf.custom(
            lambda epid: isinstance(test_utils.to_string(epid), str)
            and len(test_utils.to_string(epid))==32 if epid else False,
            'executor_profile_id'
        ),
        nf.custom(
            lambda ptid: isinstance(test_utils.to_string(ptid), str)
            and len(test_utils.to_string(ptid))==32 if ptid else False,
            'park_taximeter_id'
        ),
    )
    .project(
        'app_version_name',
        'device_manufacturer', 'device_model', 'device_type', 'os_version',
        'event', 'event_name', 'event_type', 'event_value', 'scenario', 'screen', 'utc_event_dttm',
        'lat', 'lon', 'region_id',
        'order_id', 
        'executor_profile_id', 'park_taximeter_id',
        'session_type',
        executor_unique_id=ne.custom(
            lambda epid, ptid: test_utils.to_string(ptid) + '_' + test_utils.to_string(epid),
            'executor_profile_id', 'park_taximeter_id',
        ),
        utc_event_dt=ne.custom(
            lambda x: str(datetime.datetime.strptime(test_utils.to_string(x), '%Y-%m-%d %H:%M:%S').date()),
            'utc_event_dttm',
        ),
    )
    .groupby('executor_unique_id', 'utc_event_dt',)
    .sort('utc_event_dttm')
    .reduce(executor_reducer, intensity='ultra_cpu', memory_limit=8192)
    .join(
        logistics_couriers,
        type='left',
        by_left=('executor_unique_id', 'utc_event_dt'),
        by_right=('dbid_uuid', 'utc_dt'),
        assume_unique=True,
        assume_defined=True,
        allow_undefined_keys=False,
        memory_limit=8192,
    )
    .put(PROJECT_PATH + 'executors_mart_{}_{}'.format(DT_START, DT_END))
)

In [ ]:
# job.flow_graph

In [ ]:
job.run()

### Executors mart

In [ ]:
executors_mart = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/product/postpayment/executors_mart_2021-04-26_2021-05-13')
    .as_dataframe()
)

In [ ]:
executors_mart.head()

In [ ]:
active_region_executors = set(
    executors_mart[executors_mart['active_test_region']]['executor_unique_id']
)

In [ ]:
executors_mart_mini = executors_mart[
    executors_mart['executor_unique_id'].isin(active_region_executors)
]

In [ ]:
executors_list = []
for executor_id, executor_df in tqdm.notebook.tqdm(executors_mart_mini.groupby('executor_unique_id')):
    executor_dict = {'executor_id': executor_id}
    executor_df = executor_df.sort_values('utc_event_dt')
    
    # General stats.
    executor_dict['active_in_region_flg'] = (executor_df['active_test_region']==True).sum() > 0
    executor_dict['logistics_courier_flg'] = (executor_df['logistics_courier']==True).sum() > 0
    executor_dict['correct_app_version_flg'] = executor_df['correct_app_version_flg'].iloc[-1]
    executor_dict['correct_android_version_flg'] = executor_df['correct_android_version_flg'].iloc[-1]
    # Exam stats.
    executor_dict['seen_exam_flg'] = (executor_df['seen_exam_flg']==True).sum() > 0
    executor_dict['opened_exam_flg'] = (executor_df['opened_exam_flg']==True).sum() > 0
    executor_dict['switched_option_on_flg'] = (executor_df['switched_option_on_flg']==True).sum() > 0
    executor_dict['no_nfc_screen_shown_flg'] = (executor_df['no_nfc_screen_shown_flg']==True).sum() > 0
    executor_dict['no_nfc_button_clicked_flg'] = (executor_df['no_nfc_button_clicked_flg']==True).sum() > 0
    executor_dict['no_tap2go_screen_shown_flg'] = (executor_df['no_tap2go_screen_shown_flg']==True).sum() > 0
    executor_dict['no_tap2go_button_clicked_flg'] = (executor_df['no_tap2go_button_clicked_flg']==True).sum() > 0
    
    active_dates = {'2021-05-12', '2021-05-13'}
    executor_dict['active_dates_flg'] = (executor_df['utc_event_dt'].isin(active_dates)).sum() > 0
    
    executors_list.append(executor_dict)
    

In [ ]:
executors_funnel_df = pd.DataFrame(executors_list)

In [ ]:
executors_funnel_df.head()

In [ ]:
metrics_dict = {}

metrics_dict['active_cnt'] = executors_funnel_df.shape[0]

active_logistics_couriers = executors_funnel_df[
    (executors_funnel_df['logistics_courier_flg'])
#     & (executors_funnel_df['active_dates_flg'])
]
metrics_dict['active_logistics_couriers_cnt'] = active_logistics_couriers.shape[0]

correct_android = active_logistics_couriers[
    active_logistics_couriers['correct_android_version_flg']
]
metrics_dict['correct_android_cnt'] = correct_android.shape[0]

correct_app = correct_android[
    correct_android['correct_app_version_flg']
]
metrics_dict['correct_app_cnt'] = correct_app.shape[0]

seen_exam = correct_app[correct_app['seen_exam_flg']]
metrics_dict['seen_exam_cnt'] = seen_exam.shape[0]

opened_exam = seen_exam[seen_exam['opened_exam_flg']]
metrics_dict['opened_exam_cnt'] = opened_exam.shape[0]

switched_option_on = opened_exam[opened_exam['switched_option_on_flg']]
metrics_dict['switched_option_on_cnt'] = switched_option_on.shape[0]

no_nfc_screen_shown = switched_option_on[switched_option_on['no_nfc_screen_shown_flg']]
metrics_dict['no_nfc_screen_shown_cnt'] = no_nfc_screen_shown.shape[0]

no_nfc_button_clicked = no_nfc_screen_shown[no_nfc_screen_shown['no_nfc_button_clicked_flg']]
metrics_dict['no_nfc_button_clicked_cnt'] = no_nfc_button_clicked.shape[0]

no_tap2go_screen_shown = switched_option_on[switched_option_on['no_tap2go_screen_shown_flg']]
metrics_dict['no_tap2go_screen_shown_cnt'] = no_tap2go_screen_shown.shape[0]

no_tap2go_button_clicked = no_tap2go_screen_shown[no_tap2go_screen_shown['no_tap2go_button_clicked_flg']]
metrics_dict['no_tap2go_button_clicked_cnt'] = no_tap2go_button_clicked.shape[0]

In [ ]:
metrics_dict

### Call executors

In [ ]:
seen_exam[
    ~(seen_exam['opened_exam_flg'])
    & ~(seen_exam['switched_option_on_flg'])
]

In [ ]:
switched_option_on_set = set(switched_option_on['executor_id'])
did_not_switch_on_set = set(seen_exam['executor_id']).difference(switched_option_on_set)

In [ ]:
did_not_switch_on_df = pd.DataFrame(did_not_switch_on, columns=['dbid_uuid'])
did_not_switch_on_df['postpayment_switch_on_flg'] = False

switch_on_df = pd.DataFrame(switched_option_on_set, columns=['dbid_uuid'])
switch_on_df['postpayment_switch_on_flg'] = True

pp_switch_df = pd.concat([did_not_switch_on_df, switch_on_df])

In [ ]:
len(did_not_switch_on_set)

In [ ]:
len(switched_option_on_set)

In [ ]:
# cluster.write(
#     '//home/taxi-delivery/analytics/dev/product/postpayment/pp_switch_executors_2021-04-26_2021-05-11',
#     pp_switch_df
# )

In [ ]:
pd_ids = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/product/postpayment/pp_switch_executors_pd_ids')
    .as_dataframe()
)


In [ ]:
pd_ids['phone_pd_id'] = pd_ids['phone_pd_id_list'].apply(lambda x: x[0] if x else None)

In [ ]:
pd_ids.head()

In [ ]:
cluster.write(
    '//home/taxi-delivery/analytics/dev/product/postpayment/pp_switch_executors_pd_ids_v1',
    pd_ids
)

In [ ]:
from qb2.api.v1 import typing as qt

In [ ]:
TABLE = '//home/taxi-delivery/analytics/dev/product/postpayment/pp_switch_executors_pd_ids_v1'
SCHEMA_PYTHON = {
    'dbid_uuid': qt.type_from_simple_description(str),
    'phone_pd_id_list': qt.type_from_simple_description([str]),
    'postpayment_switch_on_flg': qt.type_from_simple_description(bool),
    'phone_pd_id': qt.type_from_simple_description(str),
}

In [ ]:
job = cluster.job()

In [ ]:
table = job.table(TABLE)

table.put(TABLE + '_schema', schema=SCHEMA_PYTHON)

In [ ]:
job.run()

### Dig specific executor

In [ ]:
executor_df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/product/postpayment/executor_events_v0')
    .as_dataframe()
)

In [ ]:
executor_df.head()

In [ ]:
# executor_df['event_name'].value_counts()

In [ ]:
executor_df.sort_values('utc_event_dttm')[executor_df['event_name']=='postpayment_ui_event']['event_value'].apply(
    lambda x: x.get('postpaymentparams').get('event_type')
)

In [ ]:
executor_df.sort_values('utc_event_dttm')[
    executor_df['event_name']=='postpayment_ui_event'
]['utc_event_dttm']

In [ ]:
post_events = executor_df[
   (executor_df['utc_event_dttm']>='2021-05-05 06:42:40')
    & (executor_df['utc_event_dttm']<='2021-05-05 07:00:00')
].sort_values('utc_event_dttm')

In [ ]:
# post_events[
#     ~pd.isnull(post_events['event_value'].apply(lambda x: x.get('uiparams')))
# ]['event_value'].apply(lambda x: x.get('uiparams'))

In [ ]:
def row_func(row):
    if set(row['event_value'].keys()) - set(['commonparams']):
        return row['event_value'].get((set(row['event_value'].keys()) - set(['commonparams'])).pop())
    return None

In [ ]:
post_events['rest_value'] = post_events.apply(
    row_func, axis=1
)

In [ ]:
# post_events

In [ ]:
for r in post_events[['event_name', 'rest_value', 'utc_event_dttm']].itertuples():
    print(r.utc_event_dttm)
    print(r.event_name)
    print(r.rest_value)
    print()

### Appmetrica events digging

In [ ]:
df['uiparams'] = df['event_value'].apply(lambda x: x.get('uiparams'))

In [ ]:
for row in df[~pd.isnull(df['uiparams'])].itertuples():
    print(row.event_value.get('diagnosticcenterparams'), row.event_value.get('uiparams'))

In [ ]:
for row in df[~pd.isnull(df['uiparams'])].itertuples():
    pprint(row.event_value)
    print()

In [ ]:
df['event_name'].value_counts()

In [ ]:
for row in df[df['event_name']=='FinalButtonClicked'].itertuples():
    pprint(row.event_value)
    print()

In [ ]:
for row in df[df['event_name']=='postpayment_ui_event'].itertuples():
    pprint(row.event_value)
    print()

In [ ]:
df_postpayment = df[
    (df['event_name']=='driver_options')
    & (df['event_value'].apply(
        lambda x: x.get('driveroptionsmetricaparams', {}).get('option_id')=='post_payment'
    ))
]

In [ ]:
df_postpayment['action'] = df['event_value'].apply(
    lambda x: x.get('driveroptionsmetricaparams', {}).get('action')
)
df_postpayment['option_value'] = df['event_value'].apply(
    lambda x: x.get('driveroptionsmetricaparams', {}).get('option_value')
)

In [ ]:
df_postpayment['action'].value_counts()

In [ ]:
df_postpayment[
    (df_postpayment['action'] == 'switch')
    & (df_postpayment['option_value']==True)
]

In [ ]:
df_postpayment[
    df_postpayment['executor_profile_id']=='421bae5c0ee346ab89ac72d5bbf94089'
].sort_values('utc_event_dttm')

### Phone models

In [ ]:
df_executors = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/product/postpayment/executor_device_models')
    .as_dataframe()
)

In [ ]:
df_executors.head()

In [ ]:
df_models = (
    df_executors
    .groupby(['device_manufacturer', 'device_model'])
    .count()[['dbid_uuid']]
    .sort_values('dbid_uuid', ascending=False)
).reset_index()

In [ ]:
df_models['fraction'] = df_models['dbid_uuid']/df_models['dbid_uuid'].sum()
df_models['cum_fraction'] = np.cumsum(df_models['fraction'])

In [ ]:
df_models

In [ ]:
# df_models.to_excel(
#     './outputs/executors_device_models_{}_{}.xlsx'
#     .format('2021-04-23', '2021-04-29')
# )